In [17]:
import pandas as pd
import numpy as np
import re
from openpyxl import load_workbook
import gc
import glob
import warnings
from pandas import ExcelWriter
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [18]:
BOMfield_mapping = pd.read_excel('./V15 BOM-RuleTable Mapping 18th Aug 2020.xlsx', \
                                 usecols=['Worksheet Key', 'Condition', 'BOM Fields', 'Rule Table Field'])
BOMfield_mapping.fillna('', inplace=True)

In [19]:
LinkedQuestionnaireField_mapping = pd.read_excel('./Linked Questionnaire Field Mapping.xlsx')
LinkedQuestionnaireField_mapping['Linked worksheet'] = LinkedQuestionnaireField_mapping['Linked worksheet'].str.strip()
LinkedQuestionnaireField_mapping['Controller'] = LinkedQuestionnaireField_mapping['Controller'].str.strip()

In [20]:
entity_mapping = pd.read_excel('./Entity Mapping.xlsx')

In [21]:
entityName = 'SG'

In [22]:
wholeFile = pd.read_excel(glob.glob('./Data/{}/InterviewWithQuestionnaires_*.xls*'.format(entityName))[0], \
                          sheet_name=None, header=None)


parsedResultData = {}
for key in wholeFile.keys():
    if key not in ['Interview', 'Table of Contents', 'Available Reference Values', 'Overrides']:
        parsedResultData[wholeFile[key].iloc[0,0].split(':')[1].replace(', Class', '').strip()] = wholeFile[key]
    else:
        parsedResultData[key] = wholeFile[key]

worksheetMapping = {}
for key in wholeFile.keys():
    if key not in ['Interview', 'Table of Contents', 'Available Reference Values', 'Overrides']:
        worksheetMapping[wholeFile[key].iloc[0,0].split(':')[1].replace(', Class', '').strip()] = key
    else:
        worksheetMapping[key] = key
        
del wholeFile
gc.collect()

22561

In [23]:
def identify_table_beginnings(df):
    '''
    Function to identify from which row each of the 3 main tables start
    '''
    table_beginning_rows_dict = {}
    row = 0
    QnA_table_beginning_Found = False
    while (QnA_table_beginning_Found == False) & (row < df.shape[0]):
        if str(df.iloc[row, 0]).find('Underwriter Comments') != -1:
            QnA_table_beginning_Found = True
        else:
            row += 1
    table_beginning_rows_dict['QnA_table_beginning_row'] = np.where(row == df.shape[0], 'None', row).flat[0]
    
    table_beginning_rows_dict['Controller_table_beginning_rows'] = []
    for i in range(df.shape[0]):
        if  str(df.iloc[i, 0]).find('Controller') != -1:
            table_beginning_rows_dict['Controller_table_beginning_rows'].append(i)
            
    table_beginning_rows_dict['Rules_table_beginning_rows'] = []
    for i in range(df.shape[0]):
        if  (str(df.iloc[i, 0]).find('Rules') != -1) & (str(df.iloc[i, 0]).find('Controller') == -1):
            table_beginning_rows_dict['Rules_table_beginning_rows'].append(i)
        
    return table_beginning_rows_dict


def create_QuestionType_FilterType(inputString):
    '''
    Helper function to separate Question type and Filter type values into two separate columns
    '''
    value = inputString.values[0].split(',', maxsplit=1)
    QuestionType = value[0].replace('Question Type = ', '')
    if len(value) > 1:
        FilterType = value[1].replace('Filter Type = ', '')
    else:
        FilterType = None
    return QuestionType, FilterType


def ranking(x):
    '''
    Helper function to initiate ranks based on 1st level of Question
    '''
    d = {}
    for i in x.columns[1:]:
        d[i + 100] = pd.factorize(x[i])[0]
        d.update()
    return pd.Series(d, index=[j + 100 for j in x.columns[1:]])


def transform(x, rcol, col):
    '''
    Helper function to generate question-answers, question types, numbers and parents, assumes for now max 6 levels of questions
    '''
    i = 103
    if len(rcol) == 1:
        x['Question'] = x[0]
        x['Question Type'] = x[1]
        x['Answer'] = x[2]
    elif len(rcol) == 2:
        x['Question'] = x.apply(lambda y: y[0] if y[i] == 0 else y[i - 100], axis=1)
        x['Question Type'] = x.apply(lambda y: y[1] if y[i] == 0 else y[i - 100 + 1], axis=1)
        x['Answer'] = x.apply(lambda y: y[2] if y[i] == 0 else y[i - 100 + 2], axis=1)
    elif len(rcol) == 3:
        x['Question'] = x.apply(lambda y: y[0] if y[i] == 0 else (y[i - 100] if y[i + 3] == 0 else y[i - 100 + 3]),
                                axis=1)
        x['Question Type'] = x.apply(
            lambda y: y[1] if y[i] == 0 else (y[i - 100 + 1] if y[i + 3] == 0 else y[i - 100 + 3 + 1]), axis=1)
        x['Answer'] = x.apply(
            lambda y: y[2] if y[i] == 0 else (y[i - 100 + 2] if y[i + 3] == 0 else y[i - 100 + 3 + 2]), axis=1)
    elif len(rcol) == 4:
        x['Question'] = x.apply(lambda y: y[0] if y[i] == 0 else (
            y[i - 100] if y[i + 3] == 0 else (y[i - 100 + 3] if y[i + 6] == 0 else y[i - 100 + 6])), axis=1)
        x['Question Type'] = x.apply(lambda y: y[1] if y[i] == 0 else (
            y[i - 100 + 1] if y[i + 3] == 0 else (y[i - 100 + 3 + 1] if y[i + 6] == 0 else y[i - 100 + 6 + 1])), axis=1)
        x['Answer'] = x.apply(lambda y: y[2] if y[i] == 0 else (
            y[i - 100 + 2] if y[i + 3] == 0 else (y[i - 100 + 3 + 2] if y[i + 6] == 0 else y[i - 100 + 6 + 2])), axis=1)
    elif len(rcol) == 5:
        x['Question'] = x.apply(lambda y: y[0] if y[i] == 0 else (
            y[i - 100] if y[i + 3] == 0 else (y[i - 100 + 3] if y[i + 6] == 0 else (y[i - 100 + 6] if y[i + 9] == 0 else y[i - 100 + 9]))), axis=1)
        x['Question Type'] = x.apply(lambda y: y[1] if y[i] == 0 else (
            y[i - 100 + 1] if y[i + 3] == 0 else (y[i - 100 + 3 + 1] if y[i + 6] == 0 else (y[i - 100 + 6 + 1] if y[i + 9] == 0 else y[i - 100 + 9 + 1]))), axis=1)
        x['Answer'] = x.apply(lambda y: y[2] if y[i] == 0 else (
            y[i - 100 + 2] if y[i + 3] == 0 else (y[i - 100 + 3 + 2] if y[i + 6] == 0 else (y[i - 100 + 6 + 2] if y[i + 9] == 0 else y[i - 100 + 9 + 2]))), axis=1)

    x['Question'] = x['Question'].str.replace('<<<[0-9]*>>>', '')
    x['Question Number'] = x[rcol].apply(lambda x: '.'.join(x.values.astype(str)), axis=1).str.replace('.0', '', regex=False)
    x['Parent'] = x['Question Number'].apply(lambda x: x.rsplit('.', 1)[0])
    final_tdf = x.drop(rcol, axis=1).drop(col, axis=1)
    return final_tdf


def rankDF(source):
    '''
    Helper function to split question-answers and rank hierarchies
    '''
    # Selecting and splitting 'Question and Answers' from source
    splitSource = pd.DataFrame(source['Questions and Answers'].str.split('    ', expand=True))
    obj_columns = list(splitSource.select_dtypes(include=['object']).columns.values)
    splitSource[1000] = '<<<' + splitSource.index.astype(str) + '>>>'
    qColList = [y for y in splitSource.columns if y % 3 == 0]
    for i in qColList:
        splitSource[i] = splitSource[i].combine(splitSource[1000], lambda a, b: b + a if a != '' and a is not None else a)
    splitSource = splitSource.drop(1000, axis=1)
    splitSource[obj_columns] = splitSource[obj_columns].replace([None], '0').replace("", np.NaN).fillna(method='ffill')
    splitSource['row'] = splitSource.index

    # Generating Rank for Question Hierarchy
    rankColList = [j + 100 for j in qColList]
    colList = [x for x in range(len(qColList) * 3)]
    qDF = splitSource[qColList].drop_duplicates()
    qDF.reset_index(drop=True, inplace=True)
    qRankDF = pd.DataFrame()
    if len(qColList) > 1:
        for i in qColList:
            if i == 0:
                qRankDF[100] = qDF[0].factorize()[0] + 1
            else:
                groupList = qColList[:i // 3]
                qRankDF[i + 100] = qDF.groupby(groupList, sort=False).apply(ranking)[i + 100].explode().reset_index(
                    drop=True)
        rankedDf = pd.concat([qDF, qRankDF], axis=1)
    else:
        qDF[100] = qDF.index + 1
        rankedDf = qDF

    # merging Question Ranks with data
    mid_df = pd.merge(splitSource, rankedDf, on=qColList)

    # Adding columns to make Dataframe consistent.
    for z in colList:
        if z not in mid_df.columns:
            mid_df[z] = 0
    mid_df = mid_df.astype(object)
    return mid_df, rankColList, colList


def create_QnA_table(key, data, table_beginning_rows_dict):
    '''
    Function to create a dataframe for the QnA part, assumes for now that there are always 6 columns
    '''
    if len(table_beginning_rows_dict['Controller_table_beginning_rows']) > 0:
        table_end_row = np.int8(table_beginning_rows_dict['Controller_table_beginning_rows'][0]) - 1
    elif len(table_beginning_rows_dict['Rules_table_beginning_rows']) > 0:
        table_end_row = np.int8(table_beginning_rows_dict['Rules_table_beginning_rows'][0]) - 1
    else:
        table_end_row = data.shape[0] + 1

    if key == 'Interview':
        source = data.iloc[np.int8(table_beginning_rows_dict['QnA_table_beginning_row']) + 1: table_end_row, 0:7]
        source.columns = ['Underwriter Comments', 'Section Name', 'Key Name', 'BOM Fields', 'Questions and Answers',
                          'DisclosureType and Questionnaire', 'Help Text']
        source.drop(['Section Name'], axis=1, inplace=True)
    else:
        source = data.iloc[np.int8(table_beginning_rows_dict['QnA_table_beginning_row']) + 1: table_end_row, 0:6]
        source.columns = ['Underwriter Comments', 'Key Name', 'BOM Fields', 'Questions and Answers',
                          'DisclosureType and Questionnaire', 'Help Text']

    source.dropna(inplace=True, how='all')
    source = source.astype(object)
    #     df.fillna('', inplace=True)
    source.reset_index(drop=True, inplace=True)
    disclosureCond = source['DisclosureType and Questionnaire'].str.contains('Disclosure Type = ') & source[
        'DisclosureType and Questionnaire'].str.contains('Questionnaire = ')
    if source[(disclosureCond)].empty:
        source['Disclosure Type'] = source['DisclosureType and Questionnaire']
        source['Questionnaire'] = np.NaN
    else:
        source[['Disclosure Type', 'Questionnaire']] = source['DisclosureType and Questionnaire'].str.split(',',
                                                                                                            expand=True)

    fsource = source.drop('DisclosureType and Questionnaire', axis=1)

    # finding MIN and MAX for Numeric Question
    minMaxCond = fsource['Questions and Answers'].str.contains('Minimum = ') & fsource[
        'Questions and Answers'].str.contains('Maximum = ')
    if fsource[minMaxCond].empty:
        fsource['Min'] = np.NaN
        fsource['Max'] = np.NaN
    else:
        fsource[['Min', 'Max']] = fsource[minMaxCond]['Questions and Answers'].str.replace('\s{1,}=\s{1,}',
                                                                                           '=').str.strip().str.split(
            ' ', 1, expand=True)
    fsource['Classes'] = fsource[(fsource['Questions and Answers'].str.contains('Classes = '))]['Questions and Answers']
    fsource['Available Units'] = fsource[(fsource['Questions and Answers'].str.contains('Available Units = '))][
        'Questions and Answers']

    clist = fsource.index[
        fsource['Min'].notnull() | fsource['Classes'].notnull() | fsource['Available Units'].notnull()].tolist()
    for i in clist:
        fsource.at[i - 2, ['Min', 'Max', 'Classes', 'Available Units']] = fsource.loc[
            i, ['Min', 'Max', 'Classes', 'Available Units']]

    rdf, rcol, col = rankDF(fsource)
    tdf = transform(rdf, rcol, col)
    filteredDf = tdf[(tdf['Question Type'].str.contains("Question Type"))].copy()
    filtered0Df = pd.concat(
        [filteredDf.groupby('Question Number').filter(lambda x: len(x) == 1),
         filteredDf.groupby('Question Number').filter(lambda x: len(x) > 1)[
             filteredDf.groupby('Question Number').filter(lambda x: len(x) > 1).Answer != '0']]
    ).sort_values('Question Number')

    jrdf = pd.merge(tdf, fsource, left_on=tdf.row, right_on=fsource.index)
    jrdf = jrdf.drop(['key_0', 'row'], axis=1)

    col = ['Question', 'Questions and Answers', 'Underwriter Comments', 'Key Name', 'BOM Fields', 'Help Text',
           'Disclosure Type', 'Questionnaire', 'Min', 'Max', 'Classes', 'Available Units']
    pdf = jrdf[col[2:]].dropna(axis=0, how='all')
    BOM_df = pdf.merge(jrdf[col[:2]], left_index=True, right_index=True, how='left')
    BOM_df['Questions and Answers'] = BOM_df['Questions and Answers'].str.strip()
    qdf = BOM_df[(BOM_df['Question'].str.strip() == BOM_df['Questions and Answers'])].drop('Questions and Answers',
                                                                                           axis=1).merge(filtered0Df,
                                                                                                         on='Question').rename(
        columns={"Key Name": "Question Key Name", "BOM Fields": "Question BOM Fields"})
    adf = BOM_df[(BOM_df['Question'].str.strip() != BOM_df['Questions and Answers'])].merge(filtered0Df,
                                                                                            right_on=['Question',
                                                                                                      'Answer'],
                                                                                            left_on=['Question',
                                                                                                     'Questions and Answers']).drop(
        'Questions and Answers', axis=1).rename(
        columns={"Key Name": "Answer Key Name", "BOM Fields": "Answer BOM Fields"})
    qadf = pd.concat([qdf, adf]).groupby('row').apply(
        lambda x: x.fillna(method='ffill').fillna(method='bfill').drop_duplicates())
    edf = filtered0Df[(filtered0Df['row'].isin(qadf['row']) == False)]
    QnA_df = pd.concat([edf, qadf]).sort_values(['row']).drop('row', axis=1)
    QnA_df.reset_index(drop=True, inplace=True)
    QnA_df = QnA_df.astype(str)
    #Cleansing
    QnA_df[['Question Type', 'Filter Type']] = QnA_df[['Question Type']]\
                                                .apply(create_QuestionType_FilterType, axis=1, result_type='expand')
    QnA_df['Question Number'] = QnA_df['Question Number'].str.replace('.0', '', regex=False)
    QnA_df['Parent'] = QnA_df['Parent'].str.replace('.0', '', regex=False)
    QnA_df['Answer'] = QnA_df['Answer'].str.replace('0', 'nan')
    QnA_df['Min'] = QnA_df['Min'].str.replace('Minimum=', '')
    QnA_df['Max'] = QnA_df['Max'].str.replace('Maximum=', '')
    QnA_df['Classes'] = QnA_df['Classes'].str.replace('Classes = ', '')
    QnA_df['Available Units'] = QnA_df['Available Units'].str.replace('Available Units = ', '')
    QnA_df['Disclosure Type'] = QnA_df['Disclosure Type'].str.replace('Disclosure Type = ', '')
    QnA_df['Questionnaire'] = QnA_df['Questionnaire'].str.replace('Questionnaire = ', '')
    QnA_df['Version'] = 15
    QnA_df['Entity'] = entity_mapping.loc[entity_mapping['EntityName']==entityName]['EntityNumber'].values[0]
    QnA_df['Condition'] = conditionMapping_df.loc[conditionMapping_df['ConditionName'] == key]['ConditionNumber'].values[0]
    QnA_df['Unique Key'] = QnA_df['Version'].astype(str) + QnA_df['Entity'].astype(str) + QnA_df['Condition'].astype(str) + \
                            QnA_df['Question Number']
    QnA_df = QnA_df.replace('nan', '').replace('None', '')
    QnA_df = QnA_df[['Unique Key', 'Version', 'Entity', 'Condition', 'Question Number', 'Parent', 'Underwriter Comments', \
                     'Question Key Name', 'Question BOM Fields', 'Answer Key Name', 'Answer BOM Fields', \
                     'Question', 'Question Type', 'Filter Type', 'Min', 'Max',\
                     'Answer', 'Disclosure Type', 'Questionnaire', 'Classes', 'Available Units', 'Help Text']]
    return QnA_df


def splitRows(df, colToSplitOn):
    '''
    Helper function to split rows of a dataframe based on a particular column and replicate all other columns
    '''
    temp = df[colToSplitOn].str.rstrip('\n').str.split('\n')
    df = df.reindex(df.index.repeat(temp.apply(len)))
    df[colToSplitOn] = np.hstack(temp)
    return df


def create_ControllerAndRules_tables(data, table_beginning_rows_dict, table):
    '''
    Function to create dataframe for the Controller/Rules table as required after identifying rows and columns and splitting
    '''
    if table == 'Controller': 
        if len(table_beginning_rows_dict['Controller_table_beginning_rows']) == 0:
            return None
        else:
            df_dict = {}
            for i in range(len(table_beginning_rows_dict['Controller_table_beginning_rows'])):
                table_beginning_row = np.int32(table_beginning_rows_dict['Controller_table_beginning_rows'][i])
                if i == len(table_beginning_rows_dict['Controller_table_beginning_rows']) - 1:
                    if len(table_beginning_rows_dict['Rules_table_beginning_rows']) == 0:
                        table_end_row = data.shape[0]
                    else:
                        table_end_row = np.int32(table_beginning_rows_dict['Rules_table_beginning_rows'][0]) - 1
                else:
                    table_end_row = np.int32(table_beginning_rows_dict['Controller_table_beginning_rows'][i+1]) - 1
                col = 0
                table_end_Found = False
                while (table_end_Found == False) & (col < data.shape[1]):
                    if pd.isna(data.iloc[table_beginning_row + 1, col]) == True:
                        table_end_Found = True
                    else:
                        col += 1  
                df = data.iloc[table_beginning_row+2:table_end_row+1, 0:col]
                df.columns = data.iloc[table_beginning_row+1, 0:col].tolist()
                                            
                df.dropna(inplace=True, how='all')
                df.fillna('', inplace=True)
                df.reset_index(drop=True, inplace=True)
                df.columns = [re.sub(' +',' ',col) for col in df.columns]
                if 'Product Class' not in df.columns:
                    df['Product Class'] = ''
                df = splitRows(df, 'Product Class')
                df_dict[i] = df
            return df_dict

    elif table == 'Rules':
        if len(table_beginning_rows_dict['Rules_table_beginning_rows']) == 0:
            return None
        else:
            table_beginning_row = np.int32(table_beginning_rows_dict['Rules_table_beginning_rows'][0])
            table_end_row = data.shape[0] + 1
            col = 0
            table_end_Found = False
            while (table_end_Found == False) & (col < data.shape[1]):
                if pd.isna(data.iloc[table_beginning_row + 1, col]) == True:
                    table_end_Found = True
                else:
                    col += 1  

            df = data.iloc[table_beginning_row+2:table_end_row, 0:col]
            df.columns = data.iloc[table_beginning_row+1, 0:col].tolist()
            df.dropna(inplace=True, how='all')
            df.fillna('', inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.columns = [re.sub(' +',' ',col) for col in df.columns]
            df.columns = [col.replace('SinceLast', 'Since Last') if 'SinceLast' in col else col for col in df.columns]
            if 'Product Class' not in df.columns:
                df['Product Class'] = ''
            df = splitRows(df, 'Product Class')
            return df


In [24]:
def createFinalOutput(sheetKey, table_beginning_rows_dict, QnA_df, Controller_df_dict, Rules_df):
    if sheetKey == 'Interview':
        ControllerTableRows_df = pd.DataFrame()
        for i in range(QnA_df.shape[0]):
            key = sheetKey
            if (QnA_df['Questionnaire'].iloc[i] != '') & (QnA_df['Disclosure Type'].iloc[i] != ''):
                disclosureType = QnA_df['Disclosure Type'].iloc[i]
                if QnA_df['Questionnaire'].iloc[i].find('Overrides') == -1:
                    questionnaire = QnA_df['Questionnaire'].iloc[i].replace(' - Default', '').strip()
                if (questionnaire is not None) & (questionnaire not in neither_Rules_nor_Controller) & \
                    (LinkedQuestionnaireField_mapping.loc[LinkedQuestionnaireField_mapping['Linked worksheet'] \
                                                                     ==questionnaire].shape[0] > 0):
                    fieldName = LinkedQuestionnaireField_mapping.loc[LinkedQuestionnaireField_mapping['Linked worksheet'] \
                                                                     ==questionnaire]['Controller'].values[0]
                    for dict_key in newData[questionnaire]['Controller_df_dict'].keys():
                        temp = newData[questionnaire]['Controller_df_dict'][dict_key]
                        if 'Risk Name' in temp.columns:
                            condition = ((temp[fieldName] == disclosureType) | (temp['Risk Name'] == disclosureType))
                        else:
                            condition = (temp[fieldName] == disclosureType)
                        if temp.loc[condition].shape[0] > 0:
                            temp_Controller = temp.loc[condition]
                            temp_Controller['Row_Number'] = QnA_df.index[i]
                            temp_Controller['Controller Table Field'] = fieldName
                            ControllerTableRows_df = pd.concat([ControllerTableRows_df, temp_Controller]).fillna('')
                            break
        ControllerTableRows_df.columns = [col if col == 'Controller Table Field' else 'C - ' + col for col in ControllerTableRows_df.columns]
        if ControllerTableRows_df.shape[0] > 0:
            final_df = QnA_df.merge(ControllerTableRows_df, left_index=True, right_on='C - Row_Number', how='left')
            final_df.drop(['C - Row_Number'] ,axis=1, inplace=True)
            final_df.fillna('', inplace=True)
            final_df.fillna('', inplace=True)
        else:
            final_df = QnA_df
        return final_df
    
    elif sheetKey in neither_Rules_nor_Controller or sheetKey in Controller_but_no_Rules:
        conditionName = sheetKey.split('-')[1].strip()
        ControllerTableRows_df = pd.DataFrame()
        for i in range(QnA_df.shape[0]):
            key = sheetKey
            if (QnA_df['Questionnaire'].iloc[i] != '') & (QnA_df['Disclosure Type'].iloc[i] != ''):
                disclosureType = QnA_df['Disclosure Type'].iloc[i]
                if QnA_df['Questionnaire'].iloc[i].find('Overrides') == -1:
                    questionnaire = QnA_df['Questionnaire'].iloc[i].replace(' - Default', '').strip()
                elif QnA_df['Questionnaire'].iloc[i].find('Overrides') != -1:
                    if overrideMapping_df.loc[(overrideMapping_df['Main DT'] == disclosureType) & \
                                                           (overrideMapping_df['Associated DT'] == conditionName)].shape[0] > 0:
                        questionnaire = overrideMapping_df.loc[(overrideMapping_df['Main DT'] == disclosureType) & \
                                                           (overrideMapping_df['Associated DT'] == conditionName)] \
                                                            ['Override Questionnaire'].values[0]
                    else:
                        questionnaire = None

                if (questionnaire is not None) & (questionnaire not in neither_Rules_nor_Controller) & \
                    (LinkedQuestionnaireField_mapping.loc[LinkedQuestionnaireField_mapping['Linked worksheet'] \
                                                                     ==questionnaire].shape[0] > 0):
                    fieldName = LinkedQuestionnaireField_mapping.loc[LinkedQuestionnaireField_mapping['Linked worksheet'] \
                                                                     ==questionnaire]['Controller'].values[0]
                    for dict_key in newData[questionnaire]['Controller_df_dict'].keys():
                        temp = newData[questionnaire]['Controller_df_dict'][dict_key]
                        if 'Risk Name' in temp.columns:
                            condition = ((temp[fieldName] == disclosureType) | (temp['Risk Name'] == disclosureType))
                        else:
                            condition = (temp[fieldName] == disclosureType)
                        if temp.loc[condition].shape[0] > 0:
                            temp_Controller = temp.loc[condition]
                            temp_Controller['Row_Number'] = QnA_df.index[i]
                            temp_Controller['Controller Table Field'] = fieldName
                            ControllerTableRows_df = pd.concat([ControllerTableRows_df, temp_Controller]).fillna('')
                            break
            
            if (QnA_df['Question Type'].iloc[i] != 'Text Area Question') & \
                (QnA_df['Answer'].iloc[i] not in ['No', 'None of These']) & \
                (key in ['Aviation Experience - Flying', 'Aviation Experience - Sky Sports', 'Climbing Experience – Hiking',\
                         'Lifestyle Activity - Motor Sport', 'Underwater Diving - Scuba Diving']):
                for dict_key in newData[key]['Controller_df_dict'].keys():
                    temp_Controller = newData[key]['Controller_df_dict'][dict_key]
                    temp_Controller['Row_Number'] = QnA_df.index[i]
                    temp_Controller['Controller Table Field'] = ''
                    ControllerTableRows_df = pd.concat([ControllerTableRows_df, temp_Controller]).fillna('')
            
        ControllerTableRows_df.columns = [col if col == 'Controller Table Field' else 'C - ' + col for col in ControllerTableRows_df.columns]
        if ControllerTableRows_df.shape[0] > 0:
            final_df = QnA_df.merge(ControllerTableRows_df, left_index=True, right_on='C - Row_Number', how='left')
            final_df.drop(['C - Row_Number'] ,axis=1, inplace=True)
            final_df.fillna('', inplace=True)
            final_df.fillna('', inplace=True)
        else:
            final_df = QnA_df
        return final_df
    
    else:
        conditionName = sheetKey.split('-')[1].strip()
        ControllerTableRows_df = pd.DataFrame()
        RulesTableRows_df = pd.DataFrame()

        for i in range(QnA_df.shape[0]):
            key = sheetKey
            if QnA_df['Question BOM Fields'].iloc[i] != '':
                bomField = QnA_df['Question BOM Fields'].iloc[i]
                if BOMfield_mapping.loc[(BOMfield_mapping['Worksheet Key'] == key) & \
                                                       (BOMfield_mapping['BOM Fields'] == bomField)]['Rule Table Field'].shape[0] > 0:
                    ruleTableField = BOMfield_mapping.loc[(BOMfield_mapping['Worksheet Key'] == key) & \
                                                           (BOMfield_mapping['BOM Fields'] == bomField)]['Rule Table Field'].values[0]
                    if (ruleTableField in Rules_df.columns) & (Rules_df.shape[0] > 0):
                        if QnA_df['Disclosure Type'].iloc[i] != '':
                            disclosureType = QnA_df['Disclosure Type'].iloc[i]
                            if Rules_df.loc[Rules_df[ruleTableField] == disclosureType].shape[0] > 0:
                                temp_Rules = Rules_df.loc[Rules_df[ruleTableField] == disclosureType]
                                temp_Rules['Row_Number'] = QnA_df.index[i]
                                temp_Rules['Rules Table Field'] = ruleTableField
                                RulesTableRows_df = pd.concat([RulesTableRows_df, temp_Rules]).fillna('')
                        else:
                            if QnA_df['Question Type'].iloc[i] in ['Radio Selection', 'Check Box']:
                                if QnA_df['Answer Key Name'].iloc[i].find('drv_') != -1:
                                    keyName = QnA_df['Answer Key Name'].iloc[i].split(',')
                                    keyName = [word.strip() for word in keyName if 'drv_' in word][0]
                                    answerStringToSearch = drvMapping_df.loc[drvMapping_df['Available Reference Values'] == keyName] \
                                                                            ['Available Reference Values Display Name'].iloc[0]
                                else:
                                    answerStringToSearch = QnA_df['Answer'].iloc[i]
                                if Rules_df.loc[Rules_df[ruleTableField] == answerStringToSearch].shape[0] > 0:
                                    temp_Rules = Rules_df.loc[Rules_df[ruleTableField] == answerStringToSearch]
                                    temp_Rules['Row_Number'] = QnA_df.index[i]
                                    temp_Rules['Rules Table Field'] = ruleTableField
                                    RulesTableRows_df = pd.concat([RulesTableRows_df, temp_Rules]).fillna('')
                            elif (QnA_df['Question Type'].iloc[i] in ['Date Question', \
                                                                       'Numeric Question', 'Unitized Question']):
                                if Rules_df.loc[Rules_df[ruleTableField] != '-'].shape[0] > 0:
                                    temp_Rules = Rules_df.loc[Rules_df[ruleTableField] != '-']
                                    temp_Rules['Row_Number'] = QnA_df.index[i]
                                    temp_Rules['Rules Table Field'] = ruleTableField
                                    RulesTableRows_df = pd.concat([RulesTableRows_df, temp_Rules]).fillna('')

            if QnA_df['Answer BOM Fields'].iloc[i] != '':
                bomField = QnA_df['Answer BOM Fields'].iloc[i]
                if BOMfield_mapping.loc[(BOMfield_mapping['Worksheet Key'] == key) & \
                                       (BOMfield_mapping['BOM Fields'] == bomField)]['Rule Table Field'].shape[0] > 0:
                    ruleTableField = BOMfield_mapping.loc[(BOMfield_mapping['Worksheet Key'] == key) & \
                                                           (BOMfield_mapping['BOM Fields'] == bomField)]['Rule Table Field'].values[0]
                    if QnA_df['Disclosure Type'].iloc[i] != '':
                        disclosureType = QnA_df['Disclosure Type'].iloc[i]

                    if (ruleTableField in Rules_df.columns) & (Rules_df.shape[0] > 0):
                        if disclosureType != '':
                            if Rules_df.loc[Rules_df[ruleTableField] == disclosureType].shape[0] > 0:
                                temp_Rules = Rules_df.loc[Rules_df[ruleTableField] == disclosureType]
                                temp_Rules['Row_Number'] = QnA_df.index[i]
                                temp_Rules['Rules Table Field'] = ruleTableField
                                RulesTableRows_df = pd.concat([RulesTableRows_df, temp_Rules]).fillna('')
                        else:
                            if QnA_df['Question Type'].iloc[i] == 'Radio Selection':
                                if QnA_df['Answer Key Name'].iloc[i].find('drv_') != -1:
                                    keyName = QnA_df['Answer Key Name'].iloc[i].split(',')
                                    keyName = [word.strip() for word in keyName if 'drv_' in word][0]
                                    answerStringToSearch = drvMapping_df.loc[drvMapping_df['Available Reference Values'] == keyName] \
                                                                            ['Available Reference Values Display Name'].iloc[0]
                                else:
                                    answerStringToSearch = QnA_df['Answer'].iloc[i]
                                if Rules_df.loc[Rules_df[ruleTableField] == answerStringToSearch].shape[0] > 0:
                                    temp_Rules = Rules_df.loc[Rules_df[ruleTableField] == answerStringToSearch]
                                    temp_Rules['Row_Number'] = QnA_df.index[i]
                                    temp_Rules['Rules Table Field'] = ruleTableField
                                    RulesTableRows_df = pd.concat([RulesTableRows_df, temp_Rules]).fillna('')
                            elif (QnA_df['Question Type'].iloc[i] in ['Date Question', \
                                                                       'Numeric Question', 'Unitized Question']):
                                if Rules_df.loc[Rules_df[ruleTableField] != '-'].shape[0] > 0:
                                    temp_Rules = Rules_df.loc[Rules_df[ruleTableField] != '-']
                                    temp_Rules['Row_Number'] = QnA_df.index[i]
                                    temp_Rules['Rules Table Field'] = ruleTableField
                                    RulesTableRows_df = pd.concat([RulesTableRows_df, temp_Rules]).fillna('')

            if (QnA_df['Questionnaire'].iloc[i] != '') & (QnA_df['Disclosure Type'].iloc[i] != ''):
                disclosureType = QnA_df['Disclosure Type'].iloc[i]
                if QnA_df['Questionnaire'].iloc[i].find('Overrides') == -1:
                    questionnaire = QnA_df['Questionnaire'].iloc[i].replace(' - Default', '').strip()
                elif QnA_df['Questionnaire'].iloc[i].find('Overrides') != -1:
                    if overrideMapping_df.loc[(overrideMapping_df['Main DT'] == disclosureType) & \
                                                           (overrideMapping_df['Associated DT'] == conditionName)].shape[0] > 0:
                        questionnaire = overrideMapping_df.loc[(overrideMapping_df['Main DT'] == disclosureType) & \
                                                           (overrideMapping_df['Associated DT'] == conditionName)] \
                                                            ['Override Questionnaire'].values[0]
                    else:
                        questionnaire = None

                if (questionnaire is not None) & (questionnaire not in neither_Rules_nor_Controller) & \
                    (LinkedQuestionnaireField_mapping.loc[LinkedQuestionnaireField_mapping['Linked worksheet'] \
                                                                     ==questionnaire].shape[0] > 0):
                    fieldName = LinkedQuestionnaireField_mapping.loc[LinkedQuestionnaireField_mapping['Linked worksheet'] \
                                                                 ==questionnaire]['Controller'].values[0]
                    for dict_key in newData[questionnaire]['Controller_df_dict'].keys():
                        temp = newData[questionnaire]['Controller_df_dict'][dict_key]
                        if 'Risk Name' in temp.columns:
                            condition = ((temp[fieldName] == disclosureType) | (temp['Risk Name'] == disclosureType))
                        else:
                            condition = (temp[fieldName] == disclosureType)
                        if temp.loc[condition].shape[0] > 0:
                            temp_Controller = temp.loc[condition]
                            temp_Controller['Row_Number'] = QnA_df.index[i]
                            temp_Controller['Controller Table Field'] = fieldName
                            ControllerTableRows_df = pd.concat([ControllerTableRows_df, temp_Controller]).fillna('')
                            break
            
            #To handle anxiety specific case
            if ((QnA_df['Question Type'].iloc[i] == 'Check Box') & (QnA_df['Answer BOM Fields'].iloc[i] == '') & \
                  (QnA_df.loc[(QnA_df['Question Number'] == QnA_df['Question Number'].iloc[i]) & (QnA_df['Answer BOM Fields'] != '')].shape[0] > 0)):
                bomField = QnA_df.loc[(QnA_df['Question Number'] == QnA_df['Question Number'].iloc[i]) & \
                                      (QnA_df['Answer BOM Fields'] != '')]['Answer BOM Fields'].iloc[0]
                if bomField == 'MedicalCondition.type':
                    controllerTableField = 'Medical Type'
                    string = QnA_df['Answer'].iloc[i]
                    listToMatch = []
                    if len(re.findall(".*?\((.*?)\)", string.strip())) > 0:
                        listToMatch = re.sub(" ?\([^)]+\)", "", string.strip()).split(',') + re.findall(".*?\((.*?)\)", string.strip())[0].replace('such as', '').split(',')
                    else:
                        listToMatch = re.sub(" ?\([^)]+\)", "", string.strip()).split(',')
                    listToMatch = [l.strip().lower() for l in listToMatch]
                    listToMatch = [l.replace('generalised', 'generalized') for l in listToMatch]
                    if Controller_df_dict[0].loc[Controller_df_dict[0]['Medical Type'].str.lower().apply(lambda x: re.sub(' +',' ',x)).isin(listToMatch)].shape[0] > 0:
                        temp_Controller = Controller_df_dict[0].loc[Controller_df_dict[0]['Medical Type'].str.lower().isin(listToMatch)]
                        temp_Controller['Row_Number'] = QnA_df.index[i]
                        temp_Controller['Controller Table Field'] = controllerTableField
                        ControllerTableRows_df = pd.concat([ControllerTableRows_df, temp_Controller]).fillna('')
            
            #To handle  heart murur specific case
            if (conditionName == 'Heart Murmur') & (QnA_df['Question'].iloc[i].find('Do you have a history of any of the following?')!= -1):
                temp_Controller = Controller_df_dict[0].loc[Controller_df_dict[0]['Medical Type'] == conditionName]
                temp_Controller['Row_Number'] = QnA_df.index[i]
                temp_Controller['Controller Table Field'] = ''
                ControllerTableRows_df = pd.concat([ControllerTableRows_df, temp_Controller]).fillna('')

        RulesTableRows_df.columns = [col if col == 'Rules Table Field' else 'R - ' + col for col in RulesTableRows_df.columns]
        ControllerTableRows_df.columns = [col if col == 'Controller Table Field' else 'C - ' + col for col in ControllerTableRows_df.columns]
        if (ControllerTableRows_df.shape[0] > 0) & (RulesTableRows_df.shape[0] > 0):
            RulesControllerOutput_df = RulesTableRows_df.merge(ControllerTableRows_df, left_on=['R - Row_Number', 'R - Product Class'], \
                                    right_on=['C - Row_Number', 'C - Product Class'], how='outer')
            RulesControllerOutput_df['Row_Number'] = np.where(~RulesControllerOutput_df['R - Row_Number'].isnull(), \
                                                             RulesControllerOutput_df['R - Row_Number'], \
                                                             RulesControllerOutput_df['C - Row_Number'])
            RulesControllerOutput_df.drop(['R - Row_Number', 'C - Row_Number'], axis=1, inplace=True)

            final_df = QnA_df.merge(RulesControllerOutput_df, left_index=True, right_on='Row_Number', how='left')
            final_df.drop(['Row_Number'] ,axis=1, inplace=True)
            final_df.fillna('', inplace=True)
        elif (ControllerTableRows_df.shape[0] > 0) & (RulesTableRows_df.shape[0] == 0):
            final_df = QnA_df.merge(ControllerTableRows_df, left_index=True, right_on='C - Row_Number', how='left')
            final_df.drop(['C - Row_Number'] ,axis=1, inplace=True)
            final_df.fillna('', inplace=True)
        elif (ControllerTableRows_df.shape[0] == 0) & (RulesTableRows_df.shape[0] > 0):
            final_df = QnA_df.merge(RulesTableRows_df, left_index=True, right_on='R - Row_Number', how='left')
            final_df.drop(['R - Row_Number'] ,axis=1, inplace=True)
            final_df.fillna('', inplace=True)
        else:
            final_df = QnA_df
            final_df.fillna('', inplace=True)
        return final_df

In [25]:
conditionMapping_df = pd.DataFrame(parsedResultData['Table of Contents'].iloc[8:,3])
conditionMapping_df.columns = ['ConditionName']
conditionMapping_df = conditionMapping_df.loc[~conditionMapping_df['ConditionName'].isin(['Interview', 'Available Reference Values', 'Overrides'])]
conditionMapping_df.reset_index(drop=True, inplace=True)
conditionMapping_df['ConditionNumber'] = conditionMapping_df.index + 1
conditionMapping_df.loc[conditionMapping_df.shape[0]] = ['Interview', 9999]

In [26]:
drvMapping_df = parsedResultData['Available Reference Values'].iloc[:,:3]
drvMapping_df.columns = drvMapping_df.iloc[0,:]
drvMapping_df = drvMapping_df.iloc[1:,:]

In [27]:
overrideMapping_df = parsedResultData['Overrides'].iloc[:,:4]
overrideMapping_df.columns = overrideMapping_df.iloc[0,:]
overrideMapping_df = overrideMapping_df.iloc[1:,:]
overrideMapping_df.reset_index(drop=True, inplace=True)

dtRows = overrideMapping_df.loc[~overrideMapping_df['Main DT'].isnull()].index.tolist()
for i in range(overrideMapping_df.shape[0]):
    if i in dtRows:
        dtToReplicate = overrideMapping_df['Main DT'].iloc[i]
    else:
        overrideMapping_df['Main DT'].iloc[i] = dtToReplicate


In [28]:
newData = {}
for key in [key for key in parsedResultData.keys() if key not in ['Table of Contents', 'Available Reference Values', 'Overrides']]:
    temp_dict = {}
    temp_dict['table_beginning_rows_dict'] = identify_table_beginnings(parsedResultData[key])
    temp_dict['QnA_df'] = create_QnA_table(key, parsedResultData[key], temp_dict['table_beginning_rows_dict'])
    temp_dict['Controller_df_dict'] = create_ControllerAndRules_tables(parsedResultData[key], temp_dict['table_beginning_rows_dict'], 'Controller')
    temp_dict['Rules_df'] = create_ControllerAndRules_tables(parsedResultData[key], temp_dict['table_beginning_rows_dict'], 'Rules')
    newData[key] = temp_dict

In [29]:
Controller_but_no_Rules = []
Rules_But_no_Controller = []
neither_Rules_nor_Controller = []
both_Rules_and_Controller = []
for key in [key for key in parsedResultData.keys() if key not in ['Table of Contents', 'Available Reference Values', 'Overrides']]:
    if (newData[key]['Rules_df'] is None) & (newData[key]['Controller_df_dict'] is None):
        neither_Rules_nor_Controller.append(key)
    elif (newData[key]['Rules_df'] is None) & (newData[key]['Controller_df_dict'] is not None):
        Controller_but_no_Rules.append(key)
    elif (newData[key]['Rules_df'] is not None) & (newData[key]['Controller_df_dict'] is None):
        Rules_But_no_Controller.append(key)
    elif (newData[key]['Rules_df'] is not None) & (newData[key]['Controller_df_dict'] is not None):
        both_Rules_and_Controller.append(key)

print('# Sheets with both Controller and Rules: ', len(both_Rules_and_Controller))
print('# Sheets with Controller but no Rules: ', len(Controller_but_no_Rules))
print('# Sheets with no Controller but Rules: ', len(Rules_But_no_Controller))
print('# Sheets with neither Controller nor Rules: ', len(neither_Rules_nor_Controller))

# Sheets with both Controller and Rules:  76
# Sheets with Controller but no Rules:  31
# Sheets with no Controller but Rules:  0
# Sheets with neither Controller nor Rules:  15


In [30]:
finalData = {}
for key in [key for key in newData.keys() if key not in ['Table of Contents', 'Available Reference Values', 'Overrides']]:
    finalData[worksheetMapping[key]] = createFinalOutput(sheetKey = key,
                        table_beginning_rows_dict = newData[key]['table_beginning_rows_dict'],
                        QnA_df = newData[key]['QnA_df'],
                        Controller_df_dict = newData[key]['Controller_df_dict'],
                        Rules_df = newData[key]['Rules_df'])

In [31]:
def save_xls(dict_df, path):
    """
    Save a dictionary of dataframes to an excel file, with each dataframe as a seperate worksheet
    """
    writer = ExcelWriter(path, engine='xlsxwriter', options = {"strings_to_formulas" : False })
    for key in dict_df:
        dict_df[key].to_excel(writer, key, index=False)
    writer.save()

In [32]:
save_xls(dict_df = finalData, path = './Data/{}/PythonOutputFile.xlsx'.format(entityName))